In [1]:
#Inisiasi Model
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('job_intents.json', encoding='utf-8').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        w = nltk.word_tokenize(pattern)
        words.extend(w)

        documents.append((w, intent['tag']))


        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# inisialisasi training data

training = []
output_empty = [0] * len(classes)
for doc in documents:

    bag = []

    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)


    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

165 documents
38 classes ['info_ Bali', 'info_ DKI Jakarta', 'info_ Sulawesi Utara', 'info_Bangka Belitung', 'info_Banten', 'info_Bengkulu', 'info_DI Yogyakarta', 'info_Gorontalo', 'info_Jambi', 'info_Jawa Tengah', 'info_Jawa timur', 'info_Kalimantan Barat', 'info_Kalimantan Selatan', 'info_Kalimantan Tengah', 'info_Kalimantan Timur', 'info_Kalimantan Utara', 'info_Kepulauan Riau', 'info_Lampung', 'info_Maluku', 'info_Maluku Utara', 'info_Nanggroe Aceh Darussalam', 'info_Nusa Tenggara Barat', 'info_Nusa Tenggara Timur', 'info_Papua Barat', 'info_Riau', 'info_Sulawesi Barat', 'info_Sulawesi Selatan', 'info_Sulawesi Tengah', 'info_Sulawesi Tenggara', 'info_Sumatera Barat', 'info_Sumatera Selatan', 'info_Sumatera Utara', 'info_jawa_barat', 'name', 'options', 'perpisahan', 'salam', 'terimakasih']
76 unique lemmatized words ["'s", ',', 'about', 'aceh', 'adalah', 'anda', 'apa', 'awesome', 'bagaimana', 'bangka', 'barat', 'belitung', 'bengkulu', 'bisa', 'budaya', 'bye', 'darussalam', 'dirimu',

<ipython-input-4-9b17eb3629cf>:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [5]:
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data Dibuat")



# Membuat Model - Dengan 3 Layer. Dimana Layer pertama terdapat 128 Neurons, Layer kedua 64 neurons dan 

# Menyamakan dengan nomor intents untuk memprediksi keluaran intent dengan Softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


# Mengkompile Model. Penurunan gradien stokastik dengan gradien akselerasi Nesterov memberikan hasil yang baik untuk model ini
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

Training data Dibuat


In [6]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model telah dibuat")

Epoch 1/200
33/33 [==============================] - 0s 786us/step - loss: 3.6640 - accuracy: 0.0182
Epoch 2/200
33/33 [==============================] - 0s 778us/step - loss: 3.5695 - accuracy: 0.0606
Epoch 3/200
33/33 [==============================] - 0s 635us/step - loss: 3.4753 - accuracy: 0.0909
Epoch 4/200
33/33 [==============================] - 0s 635us/step - loss: 3.4004 - accuracy: 0.1273
Epoch 5/200
33/33 [==============================] - 0s 574us/step - loss: 3.2837 - accuracy: 0.1455
Epoch 6/200
33/33 [==============================] - 0s 551us/step - loss: 3.1466 - accuracy: 0.1818
Epoch 7/200
33/33 [==============================] - 0s 628us/step - loss: 2.9741 - accuracy: 0.2364
Epoch 8/200
33/33 [==============================] - 0s 544us/step - loss: 2.8405 - accuracy: 0.2848
Epoch 9/200
33/33 [==============================] - 0s 574us/step - loss: 2.7220 - accuracy: 0.2727
Epoch 10/200
33/33 [==============================] - 0s 574us/step - loss: 2.4685 - accura

33/33 [==============================] - 0s 574us/step - loss: 1.0051 - accuracy: 0.6788
Epoch 82/200
33/33 [==============================] - 0s 576us/step - loss: 0.9857 - accuracy: 0.6970
Epoch 83/200
33/33 [==============================] - 0s 574us/step - loss: 0.9847 - accuracy: 0.6788
Epoch 84/200
33/33 [==============================] - 0s 574us/step - loss: 0.9544 - accuracy: 0.7030
Epoch 85/200
33/33 [==============================] - 0s 574us/step - loss: 0.9447 - accuracy: 0.6788
Epoch 86/200
33/33 [==============================] - 0s 546us/step - loss: 0.9606 - accuracy: 0.6970
Epoch 87/200
33/33 [==============================] - 0s 574us/step - loss: 0.9960 - accuracy: 0.6788
Epoch 88/200
33/33 [==============================] - 0s 604us/step - loss: 0.9955 - accuracy: 0.6909
Epoch 89/200
33/33 [==============================] - 0s 604us/step - loss: 1.0030 - accuracy: 0.6727
Epoch 90/200
33/33 [==============================] - 0s 635us/step - loss: 0.9799 - accuracy: 

33/33 [==============================] - 0s 574us/step - loss: 0.9462 - accuracy: 0.6848
Epoch 161/200
33/33 [==============================] - 0s 574us/step - loss: 0.9212 - accuracy: 0.7030
Epoch 162/200
33/33 [==============================] - 0s 695us/step - loss: 0.9697 - accuracy: 0.6788
Epoch 163/200
33/33 [==============================] - 0s 574us/step - loss: 0.9797 - accuracy: 0.6727
Epoch 164/200
33/33 [==============================] - 0s 574us/step - loss: 0.9217 - accuracy: 0.7091
Epoch 165/200
33/33 [==============================] - 0s 604us/step - loss: 0.9407 - accuracy: 0.6788
Epoch 166/200
33/33 [==============================] - 0s 604us/step - loss: 0.9571 - accuracy: 0.6970
Epoch 167/200
33/33 [==============================] - 0s 604us/step - loss: 0.9258 - accuracy: 0.7030
Epoch 168/200
33/33 [==============================] - 0s 604us/step - loss: 0.9185 - accuracy: 0.7030
Epoch 169/200
33/33 [==============================] - 0s 604us/step - loss: 0.9290 - a